In [1]:
# google drive mount
from google.colab import drive
drive.mount("dataset", force_remount=True)

Mounted at dataset


In [2]:
# Pytorch install
!pip install torcheval
# create working directories
!mkdir /home/dataset
!mkdir /home/dataset/log
# copy the images into local directory
!cp -r /content/dataset/MyDrive/CNN_dataset/csi\(1c\)_AmFall_402_SDS_V0.10_O10_img_4.0  /home/dataset/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.6 MB/s eta 0:00:00


In [3]:
# v3.3.1 : for multiple processing
#
# v3.3 : save options for the test results
#   - save test result information into log directory
#
# v3.2; save options for pth
# functions v3.1, for train/validation/test images from pre-defined log files
#   - output control for chatGPT process (3.1)
#   - bug fix for filename -7 -> -6
#   - data_organize_from_files() added
#
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.io import read_image
from torchvision.transforms import ToTensor
import torch.optim as optim
from torchvision import transforms
from torchvision.models import  resnet18, ResNet18_Weights, \
                                resnet34, ResNet34_Weights, \
                                resnet50, ResNet50_Weights, \
                                efficientnet_b0, EfficientNet_B0_Weights, \
                                googlenet, GoogLeNet_Weights, \
                                vgg16, VGG16_Weights, \
                                shufflenet_v2_x0_5, ShuffleNet_V2_X0_5_Weights, \
                                vgg16_bn, VGG16_BN_Weights, \
                                inception_v3, Inception_V3_Weights

from torcheval.metrics import BinaryConfusionMatrix # pip install torcheval
from torcheval.metrics.functional import binary_f1_score, binary_accuracy, binary_precision, binary_recall, binary_confusion_matrix

import os
from os import path # path.join ('/', 'usr', 'lib') -> /usr/lib
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
import datetime

# basic utility functions

LOG_DEBUG = 0
LOG_INFO = 1
LOG_SAVE = 2
LOG_WARNING = 3
LOG_ERROR = 4
LOG_CRITICAL = 5


# logging and printing function
class CSI_log :
    #def __new__(self): # called when the call is created, __new__ -> __init__
    #
    #  return super().__new__(self)
    def __init__(self, log_level=LOG_INFO, log_buf=""): # called when the call is created, __new__ -> __init__
      self.log_buf = log_buf
      self.log_level = log_level
      self.log_results = "" # to print the summary results at the last stage (3.1)
      return super().__init__()

    def set_log_level(self, log_level):
      self.log_level = log_level

    def get_log_level(self):
      return self.log_level

    def get_log_buf(self):
      return self.log_buf

    def set_log_buf(self, log_buf):
      self.log_buf = log_buf

    def set_log_results (self, log_results):
      self.log_results = log_results

    def write (self, str, log_level=LOG_INFO, flag=True):
      if log_level >= self.log_level:
          self.log_buf += str
          if flag :
            print (str, end='')

    def result (self, str, log_level=LOG_INFO, flag=False):
      if log_level >= self.log_level:
          self.log_results += str
          if flag :
            print (self.log_results + str)

    def save(self, file_name):
      with open(file_name, 'w') as f:
        f.write(self.log_buf)
        f.write(self.log_results)
        f.close()

csi_log = CSI_log()

csi_log2 = CSI_log() # v3.3, to save the test result

# global parameters
FALL=1
NONFALL=0
#SEP=os.sep # '/'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class CSI_ImageDataset(Dataset):

    # 데이터셋의 전처리
    # img_files : full path
    # img_labels : 0 (Nonfall), 1 (Fall)
    def __init__(self, img_files, img_labels=None, ratio=0.8, train=True, \
      random_seed=42, data_read=True, transform=None, target_transform=None):
        if img_labels is None :
          img_labels = [ NONFALL if (csi_img.find ('/fall/') == -1) else FALL for csi_img in img_files]
        if ratio == 1 :
          self.img_files = img_files
          self.labels = img_labels
        else :
          x_train, x_valid, y_train, y_valid = train_test_split(img_files, img_labels, \
                      train_size=ratio, random_state=random_seed, \
                      stratify=img_labels )
          if train :
            self.img_files = x_train
            self.labels = y_train
          else :
            self.img_files = x_valid
            self.labels = y_valid

        self.data_read = data_read
        if data_read :
          self.images = [plt.imread(img) for img in self.img_files]
        self.transform = transform
        self.target_transform = target_transform

    # 총 샘플의 수
    def __len__(self):
        return len(self.labels)

    # v 3.3 modification
    # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
    def __getitem__(self, idx):
        if self.data_read : # already read
          image = self.images[idx]
          img_file = self.img_files[idx] # v3.3
        else :
          image = plt.imread(self.img_files[idx]) # self.images[idx]
          img_file = self.img_files[idx] # v3.3

        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label, img_file # v3.3

def train(dataloader, model, loss_fn, optimizer, scheduler):
    size = len(dataloader.dataset)
    model.train()
    #for batch, (X, y) in enumerate(dataloader):
    for batch, (X, y, files) in enumerate(dataloader):  # v3.3
        X, y = X.to(device), y.to(device)
        pred = model(X)
        if torch.is_tensor (pred) :
            loss = loss_fn(pred, y)
        else :
            loss = loss_fn(pred.logits, y) # for googlenet nontrained weight
        #loss = loss_fn(pred, y)
        optimizer.zero_grad() # 기울기 초기화
        loss.backward() # 역전파 (기울기 계산)
        optimizer.step() # 가중치 업데이트
    return

# global variable to save the best trained model parameters
Best_model = None
Best_loss = 1_000_000.0
Best_epoch = 0

def test(dataloader, model, loss_fn, epoch_n, best=False):
    global Best_model, Best_loss, Best_epoch

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    TP, FN, FP, TN = 0, 0, 0, 0
    accuracy, precision, recall, F1 = 0, 0, 0, 0

    with torch.no_grad(): # 리소스를  with문법을 통해 with 절 내에서만 액세스를 가능하게 하고, 블록을 나가는 경우 어떤 이유든간에 리소스를 해제
        #for X, y in dataloader: # check for (batch size) items each
        for X, y, files in dataloader: # v3.3, check for (batch size) items each
            X, y = X.to(device), y.to(device)
            if best:
              model.load_state_dict(Best_model)
              pred = model(X)
            else :
              pred = model(X)
            test_loss += loss_fn(pred, y).item()
            cm = binary_confusion_matrix(pred.argmax(1), y)
            TN += cm[0,0] # label(nonfall,0), pred(nonfall, 0)
            FP += cm[0,1] # label(nonfall,0), pred(fall,1)
            FN += cm[1,0] # label(fall, 1), pred(nonfall,0)
            TP += cm[1,1] # label(fall,1), pred(fall, 1)
            #v3.3
            preds = pred.argmax(1).cpu().numpy()
            labels = y.cpu().numpy()
            for i in range(len(preds)):
                file_name = files[i].rstrip('\n');
                csi_log2.write (f"{file_name}, {labels[i]}, {preds[i]}\n", flag=False)

    accuracy = (TP+TN) / (TP+FP+FN+TN)
    precision = TP / (TP+FP)
    recall = TP / (TP+FN)
    F1 = 2*precision*recall / (precision+recall)
    test_loss /= num_batches
    if test_loss < Best_loss : # save the best estimated model with the minimum loss
        Best_loss = test_loss
        Best_model = model.state_dict()
        Best_epoch = epoch_n
    if best :
        csi_log.write (f"*** Best model (epoch:{Best_epoch}) : ")
    global G_TP, G_FN, G_FP, G_TN
    G_TP = TP
    G_FN = FN
    G_FP = FP
    G_TN = TN
    csi_log.write (f"Avg loss({test_loss:>5f}) : ")
    csi_log.write (f"TP({TP}),FN({FN}),FP({FP}),TN({TN}) : ")
    csi_log.write (f"Accuracy({accuracy:.5f}), Precision({precision:.5f}), Recall({recall:.5f}), F1({F1:.5f}) \n")


def train_test (model, train_dl, validation_dl, test_dl, loss_fn, optimizer, scheduler):

  start_time = time.time()

  for t in range(n_epochs):
      csi_log.write(f"Epoch {t+1}, lr({optimizer.param_groups[0]['lr']}) : ")
      #if log_buf is not None :
      #  log_buf += f"Epoch {t+1}, lr({optimizer.param_groups[0]['lr']}) : "

      train(train_dl, model, loss_fn, optimizer, scheduler)
      test(validation_dl, model, loss_fn, t+1)
      #print("lr: ", optimizer.param_groups[0]['lr'])
      scheduler.step()
  #global csi_log
  csi_log.write("==========================================\n")
  csi_log.write ("Training Time: {:.4f}sec\n".format((time.time() - start_time)))

  start_time = time.time()
  test(test_dl, model, loss_fn, t+1)
  test(test_dl, model, loss_fn, t+1, best=True)

  csi_log.write ("Inference Time: {:.4f}sec\n".format((time.time() - start_time)))
  #csi_log.write( "==========================================\n")
  return


# data preparation

LOSS_CE = 0
LOSS_BCE = 1

OPT_ADAM = 0
OPT_SGDM = 1
OPT_ADAMW = 2

OPT_WD = 0.0001 # OPT_PARM for adam
OPT_MNT = 0.9   # OPT_PARM for sgdm

CNN_SNU1 = 0   # YKKIM's
CNN_SNU2 = 1  # SW's
CNN_SNU3 = 2  # SNU1 + Dropout
CNN_ENB0 = 3
CNN_ENB0_M = 4
CNN_R18 = 5
CNN_R34 = 6
CNN_GN = 7
CNN_ENB0_NPT = 8
CNN_ENB0_M_NPT = 9
CNN_R18_NPT = 10
CNN_R34_NPT = 11
CNN_GN_NPT = 12
CNN_SFNV2 = 13
CNN_SFNV2_NPT = 14
CNN_VGG16BN = 15
CNN_VGG16BN_NPT = 16

cnn_model_name = [  "SNU1",     # 0; LW-CNN in AmFall paper
                    "SNU2",     # 1
                    "SNU3",     # 2
                    "ENB0",     # 3
                    "ENB0_M",   # 4
                    "R18",      # 5
                    "R34",      # 6
                    "GN",       # 7
                    "ENB0_NPT", # 8
                    "ENB0_M_NPT", # 9
                    "R18_NPT",    # 10
                    "R34_NPT",  # 11
                    "GN_NPT",   # 12
                    "SFNV2",    # 13
                    "SFNV2_NPT", # 14
                    "VGG16BN",  # 15
                    "VGG16BN_NPT" # 16
                 ]
##############################################
# default values
n_epochs = 5
n_batch_size = 50
initial_lr = 1e-3
f_shuffle = True

# v3.3.1
G_TP = 0
G_FN = 0
G_FP = 0
G_TN = 0

# n_epochs, n_batch_size, initial_lr, f_shuffle
hyper_parm_list = [
  [ 40, 50, 1e-3, True ], # HYPER0
  [ 40, 16, 1e-3, True ], # HYPER1
  [ 40, 16, 1e-4, True ], # HYPER2
  [ 40, 50, 1e-2, True], # HYPER3
  [ 45, 50, 1e-1, True], # HYPER4
  [ 40, 50, 1e-4, True], # HYPER5
  [ 40, 50, 1e-5, True], # HYPER6
  [ 40, 16, 1e-5, True], # HYPER7
  [ 80, 50, 1e-4, True], # HYPER8
  [ 40, 50, 1e-6, True], # HYPER9
  [ 50, 50, 1e-3, True], # HYPER10
  [ 8, 60, 1e-4, False] ] # HYPERn
HYPER0 = 0
HYPER1 = 1
HYPER2 = 2
HYPER3 = 3
HYPER4 = 4
HYPER5 = 5
HYPER6 = 6
HYPER7 = 7
HYPER8 = 8
HYPER9 = 9
HYPER10 = 10

parameter_list = [
  # loss function, optimizer, optimizer parm, lr changes, lr gamma, f_shuffle
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 10, 20, 30], 0.1], # PARM0
  [LOSS_CE, OPT_ADAM, 0.001, [ 10, 20, 30], 0.1], # PARM1
  [LOSS_CE, OPT_ADAM, 0.00001, [ 10, 20, 30], 0.1], # PARM2
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 5, 10, 20, 30], 0.1], # PARM3
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 5, 10, 15, 25, 35], 0.1], # PARM4
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 20, 40, 60], 0.1], # PARM5
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 20, 30, 40], 0.1], # PARM6
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 10, 20, 30, 40 ], 0.1], # PARM7
  [LOSS_CE, OPT_ADAM, OPT_WD, [ 3, 6 ], 0.1]  # PARMn
  ]
PARM0 = 0
PARM1 = 1
PARM2 = 2
PARM3 = 3
PARM4 = 4
PARM5 = 5
PARM6 = 6
PARM7 = 7

LOSS=0      # (0) CrossEntropy, (1) BinaryCrossEntropy
OPT=1       # (0) adam, (1) sdgm
OPT_PARM=2  # weight decay (adam), momentum (sdgm)
LR_STEP=3   # learning rate changes at [n, m]
LR_GAMMA=4  # lr change rate

#root_dir = path.join('/content', 'dataset', 'MyDrive', 'CNN_dataset' )
#root_dir2 = None #path.join('/content', 'dataset', 'MyDrive', 'CNN_dataset' )
root_dir = path.join('/home', 'dataset')
root_dir2 = path.join('/content', 'dataset', 'MyDrive', 'CNN_dataset' )

TRAIN = 0   # 'csi(1c)_img8_3_seg4_1'
TEST = 1    # 'csi(1c)_img8_3_seg4_1'
S_FN=2        # model file save name
CNN=3
HYPER=4
PARM=5
R_SEED=6

##############################################
def hyper_parameter (idx=0):
  #global n_epochs, n_batch_size, initial_lr, f_shuffle
  #print(f"{hyper_parm_list[idx][0]}, {hyper_parm_list[idx][1]}, {hyper_parm_list[idx][2]}, {hyper_parm_list[idx][3]}")
  return hyper_parm_list[idx][0], hyper_parm_list[idx][1], hyper_parm_list[idx][2], hyper_parm_list[idx][3]

################################################

def loss_optimizer_scheduler (model, parm_idx=0): # hyper_parameter() should be called before this function
  parm_list = parameter_list[parm_idx]
  if parm_list[LOSS] == LOSS_CE :
    lf = nn.CrossEntropyLoss()
    csi_log.write ("Loss function (CrossEntropy) ")
  elif parm_list[LOSS] == LOSS_BCE :
    lf = nn.BCELoss()()
    csi_log.write ("Loss function (BCE) ")

  if parm_list[OPT] == OPT_ADAM :
    opt = torch.optim.Adam(model.parameters(), lr=initial_lr, weight_decay = parm_list[OPT_PARM])
    csi_log.write (f"opt (Adam, {parm_list[OPT_PARM]}) ")
  elif parm_list[OPT] == OPT_SGDM :
    opt = torch.optim.SGD(model.parameters(), lr=initial_lr, momentum = parm_list[OPT_PARM])
    csi_log.write (f"opt (Sgdm, {parm_list[OPT_PARM]}) ")
  elif parm_list[OPT] == OPT_ADAMW :
    opt = torch.optim.AdamW(model.parameters(), lr=initial_lr, weight_decay = parm_list[OPT_PARM])
    csi_log.write (f"opt (AdamW, {parm_list[OPT_PARM]}) ")

  #scheduler
  sched = optim.lr_scheduler.MultiStepLR(opt, milestones=parm_list[LR_STEP], gamma=parm_list[LR_GAMMA])
  csi_log.write (f"lr_scheduler (MultiStepLR), {parm_list[LR_STEP]}, gamma({parm_list[LR_GAMMA]})\n")
  return lf, opt, sched
################################################
# in data_organize(), the random_seed should be same for a data separation
def data_organize (e_list, data_transf, random_seed=21, check_valid=False): # hyper_parameter() should be called before this function

  if e_list[TEST][0] == '' : # test directory is same as train/validation's
    csi_img_dir = path.join(root_dir, e_list[TRAIN][0] )
    csi_img_files = path.join(csi_img_dir, e_list[TRAIN][1] )
    csi_img_files = glob.glob(csi_img_files, recursive=True)
    csi_labels = [ NONFALL if (csi_img.find ('/fall/') == -1) else FALL for csi_img in csi_img_files]

    train_valid_ds = CSI_ImageDataset(csi_img_files, csi_labels, ratio=0.8, \
                        train=True, data_read=False, random_seed=random_seed)
    test_ds = CSI_ImageDataset(csi_img_files, csi_labels, ratio=0.8, \
                        train=False, data_read=True, transform = data_transf, \
                        random_seed=random_seed)
    validation_ds = CSI_ImageDataset(train_valid_ds.img_files, train_valid_ds.labels, \
                        ratio=0.85, train=False, data_read=True, \
                        transform = data_transf, random_seed=random_seed)
    train_ds = CSI_ImageDataset(train_valid_ds.img_files, train_valid_ds.labels, \
                        ratio=0.85, train=True, data_read=True, \
                        transform = data_transf, random_seed=random_seed)

  else : # test directory is different from that of train/validation's
    # separation of train and validation
    csi_img_dir = path.join(root_dir, e_list[TRAIN][0] )
    csi_img_files = path.join(csi_img_dir, e_list[TRAIN][1] )
    csi_img_files = glob.glob(csi_img_files, recursive=True)
    csi_labels = [ NONFALL if (csi_img.find ('/fall/') == -1) else FALL for csi_img in csi_img_files]

    validation_ds = CSI_ImageDataset(csi_img_files, csi_labels, ratio=0.85, train=False, \
                        data_read=True, transform = data_transf, random_seed=random_seed)
    train_ds = CSI_ImageDataset(csi_img_files, csi_labels, ratio=0.85, train=True, \
                        data_read=True, transform = data_transf, random_seed=random_seed)

    # compose test
    csi_img_dir = path.join(root_dir, e_list[TEST][0] )
    csi_img_files = path.join(csi_img_dir, e_list[TEST][1] )
    csi_img_files = glob.glob(csi_img_files, recursive=True)
    csi_labels = [ NONFALL if (csi_img.find ('/fall/') == -1) else FALL for csi_img in csi_img_files]
    test_ds = CSI_ImageDataset(csi_img_files, csi_labels, ratio=1, train=True, \
                        data_read=True, transform = data_transf, random_seed=random_seed)

  # check if the data separation is done correctly (i.e., no duplicated file names)
  sep_result = True
  if check_valid :
    for i in range(len(train_ds)) :
        if train_ds.img_files[i] in validation_ds.img_files or train_ds.img_files[i] in test_ds.img_files :
          sep_result = False
          break
    if sep_result :
        for i in range (len(validation_ds)) :
          if validation_ds.img_files[i] in test_ds.img_files :
            sep_result = False
            break

  return train_ds, validation_ds, test_ds, sep_result

################################################

def data_organize_from_file (e_list, data_transf, file_name, search_flag=False, random_seed=21, check_valid=False): # hyper_parameter() should be called before this function
    # train_file is the name of (1C), then, the actual image files are in 35C or 70C
    # validation_file and test_file are supposed to be in a same directory
    with open(file_name, 'r') as f:
        img_lines = f.readlines()
        f.close()

    # the image file needs to have full path
    csi_img_files = []
    for img in img_lines :
        if search_flag : # for nC image directory
            img = os.path.basename(img)
            #img = img[0:len(img)-7] # 파일명은 _1.jpg 가 된다. It is a bug in 2.0
            img = img[0:len(img)-6] # 파일명은 _1.jpg 가 된다.
            #print(f"{e_list[TRAIN][1]}")
            img_dir = path.join(root_dir, e_list[TRAIN][0] )
            # updated on 01JUL24, os.path.dirname(a)
            #search_str = path.join(img_dir, '*/*/*/'+img+'*.jpg')
            search_str = path.join(img_dir, os.path.dirname(e_list[TRAIN][1]), img+'*.jpg')
            imgs = glob.glob(search_str, recursive=True)
            #print (imgs)
            csi_img_files += imgs
        else:
            # for 1C image directory, don't need to find it
            # just use it as full path file name
            #print (img.strip())
            csi_img_files.append(img.strip())
    #print (csi_img_files)
    csi_labels = [ NONFALL if (csi_img.find ('/fall/') == -1) else FALL for csi_img in csi_img_files]
    data_ds = CSI_ImageDataset(csi_img_files, csi_labels, \
                        ratio=1, train=True, data_read=True, \
                        transform = data_transf, random_seed=random_seed)
    return data_ds

################################################

class SNU1_cnn (nn.Module) : # SNU CNN module defition, by ykkim
    def __init__(self):
        super(SNU1_cnn, self).__init__()  #  생성자 제일 앞에 super() 함수를 호출하고(그렇지 않으면 에러가 발생합니다), 사용할 모듈들을 인스턴스 변수로 초기화합니다
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (8),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.fc = nn.Linear(in_features=25088, out_features=2, bias=True)
        self.softmax = nn.Softmax(dim=1)


    def forward(self, x): # forward 메서드는 입력 데이터가 주어졌을 때 선언한 인스턴스 변수(모듈)들을 통과해 출력 데이터를 만드는 메서드
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x,1) # x=torch.flatten(x,1), only batches of spatial targets supported (3D tensors) but got targets of dimension: 1
        x = self.fc(x)
        x = self.softmax(x)
        return x

class SNU3_cnn (nn.Module) : # SNU1 + Dropout layer CNN module defition, by ykkim
    def __init__(self):
        super(SNU3_cnn, self).__init__()  #  생성자 제일 앞에 super() 함수를 호출하고(그렇지 않으면 에러가 발생합니다), 사용할 모듈들을 인스턴스 변수로 초기화합니다
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (8),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d (32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.dropout = nn.Dropout(p=0.5, inplace=False) # default
        self.fc = nn.Linear(in_features=25088, out_features=2, bias=True)
        self.softmax = nn.Softmax(dim=1)


    def forward(self, x): # forward 메서드는 입력 데이터가 주어졌을 때 선언한 인스턴스 변수(모듈)들을 통과해 출력 데이터를 만드는 메서드
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x,1) # x=torch.flatten(x,1), only batches of spatial targets supported (3D tensors) but got targets of dimension: 1
        x = self.dropout(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x

class SNU2_cnn (nn.Module) : # SNU CNN module defition, by swjang
    def __init__(self):
        super(SNU2_cnn, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, stride=2) # stride = kernel size (by default)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d (128, momentum=0.5)
        )
        # Flatten will be done inbetween, in forward function
        self.layer5 = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False), # default
            nn.Linear(in_features=100352, out_features=64, bias=True),
            nn.LeakyReLU(),
            nn.Dropout(p=0.5, inplace=False), # default
            nn.Linear(in_features=64, out_features=2, bias=True)
        )
        #self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x): # forward 메서드는 입력 데이터가 주어졌을 때 선언한 인스턴스 변수(모듈)들을 통과해 출력 데이터를 만드는 메서드
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = torch.flatten(x,1) # x=torch.flatten(x,1), only batches of spatial targets supported (3D tensors) but got targets of dimension: 1
        x = self.layer5(x)
        x = self.sigmoid(x)
        return x


def cnn_model (cnn_idx=0):
  weights = None
  model = None
  global device

  if cnn_idx == CNN_ENB0 :
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights) # model = efficientnet_b0(weights=None) # TBC
    model.classifier[1] = nn.Linear(in_features=1280, out_features=2, bias=True)
  elif cnn_idx == CNN_ENB0_M :
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights) # model = efficientnet_b0(weights=None) # TBC
    model.classifier = nn.Sequential(
        nn.Linear(in_features=1280, out_features=512, bias=True),
        nn.SiLU(),
        nn.Dropout(p=0.5, inplace=False), # default
        nn.Linear(in_features=512, out_features=256, bias=True),
        nn.SiLU(),
        nn.Dropout(p=0.5, inplace=False), # default
        nn.Linear(in_features=256, out_features=2, bias=True),
        nn.SiLU()
    )
  elif cnn_idx == CNN_SNU1 : # ykkim's
    model = SNU1_cnn()
  elif cnn_idx == CNN_SNU2 : # ykkim's
    model = SNU2_cnn()
  elif cnn_idx == CNN_SNU3 : # ykkim's
    model = SNU3_cnn()
  elif cnn_idx == CNN_R18 : # R18 pretrained
    weights = ResNet18_Weights.DEFAULT
    model = resnet18 (weights=weights)
    model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
  elif cnn_idx == CNN_R34 : # R34 pretrained
    weights = ResNet34_Weights.DEFAULT
    model = resnet34 (weights=weights)
    model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
  elif cnn_idx == CNN_GN : # GN pretrained
    weights = GoogLeNet_Weights.DEFAULT
    model = googlenet (weights=weights)
    model.fc = nn.Linear(in_features=1024, out_features=2, bias=True)
  elif cnn_idx == CNN_R18_NPT : # R18 not pretrained (model only)
    #weights = ResNet18_Weights.DEFAULT # to use transforms() only
    model = resnet18 ()
    model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
  elif cnn_idx == CNN_R34_NPT : # R34 not pretrained (model only)
    #weights = ResNet34_Weights.DEFAULT # to use transforms() only
    model = resnet34 ()
    model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
  elif cnn_idx == CNN_GN_NPT : # GN not pretrained (model only)
    #weights = GoogLeNet_Weights.DEFAULT # to use transforms() only
    model = googlenet ()
    model.fc = nn.Linear(in_features=1024, out_features=2, bias=True)
  elif cnn_idx == CNN_ENB0_NPT :
    #weights = EfficientNet_B0_Weights.DEFAULT # to use transforms() only
    model = efficientnet_b0() # model = efficientnet_b0(weights=None) # TBC
    model.classifier[1] = nn.Linear(in_features=1280, out_features=2, bias=True)
  elif cnn_idx == CNN_ENB0_M_NPT :
    #weights = EfficientNet_B0_Weights.DEFAULT # to use transforms() only
    model = efficientnet_b0() # model = efficientnet_b0(weights=None) # TBC
    model.classifier = nn.Sequential(
        nn.Linear(in_features=1280, out_features=512, bias=True),
        nn.SiLU(),
        nn.Dropout(p=0.5, inplace=False), # default
        nn.Linear(in_features=512, out_features=256, bias=True),
        nn.SiLU(),
        nn.Dropout(p=0.5, inplace=False), # default
        nn.Linear(in_features=256, out_features=2, bias=True),
        nn.SiLU()
    )
  elif cnn_idx == CNN_SFNV2 : # SFNV2 pretrained
    weights = ShuffleNet_V2_X0_5_Weights.DEFAULT
    model = shufflenet_v2_x0_5 (weights=weights)
    model.fc = nn.Linear(in_features=1024, out_features=2, bias=True)
  elif cnn_idx == CNN_SFNV2_NPT : # SFNV2 not pretrained (model only)
    #weights = ShuffleNet_V2_x0_5_Weights.DEFAULT
    model = shufflenet_v2_x0_5 ()
    model.fc = nn.Linear(in_features=1024, out_features=2, bias=True)
  elif cnn_idx == CNN_VGG16BN : # VGG16BN pretrained
    weights = VGG16_BN_Weights.DEFAULT
    model = vgg16_bn (weights=weights)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)
  elif cnn_idx == CNN_VGG16BN_NPT : # VGG16BN not pretrained (model only)
    model = vgg16_bn ()
    model.classifier[6] = nn.Linear(in_features=4096, out_features=2, bias=True)

  if device == "cuda" :
    model.cuda()
  return model, weights

# v3.2, %S included
def save_model (model, s_file, root_dir, root_dir2 = None, ds_save=True, model_save=True):
  now = datetime.datetime.now()

  save_file_name = path.join(root_dir,'log', s_file) + '_' + now.strftime("%m%d%H%M%S")
  torch.save(model.state_dict(), save_file_name + '.pth')
  #global csi_log
  csi_log.write (f"saved model name : {save_file_name}\n")
  # new addition
  global G_TP, G_FN, G_FP, G_TN
  csi_log.write (f"{G_TP}, {G_FN}, {G_FP}, {G_TN}, {os.path.basename(save_file_name)}\n") # added for save convenience
  csi_log.result (f"{G_TP}\t{G_FN}\t{G_FP}\t{G_TN}\t{os.path.basename(save_file_name)}\n") # v3.1
  csi_log.write ("----------------------------------------------------\n")
  #if log_buf is not None :
  #  log_buf += local_buf + ''
  csi_log.save (save_file_name + '.log')

  if Best_model is not None:
    save_file_name = path.join(root_dir,'log', 'Best_model') + '_' + now.strftime("%m%d%H%M%S")
    if model_save :
        torch.save(Best_model, save_file_name + '_best.pth')

  if root_dir2 is not None :
    save_file_name = path.join(root_dir2, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S")
    if model_save :
        torch.save(model.state_dict(), save_file_name + '.pth')
    if Best_model is not None:
      if model_save :
          torch.save(Best_model, save_file_name + '_best.pth')
    csi_log.save (save_file_name + '.log')
  #print (local_buf+'----------------------------')

  global train_ds, validation_ds, test_ds

  if ds_save :
      train_file_str = ""
      for img in train_ds.img_files :
          train_file_str += img + '\n'
      validation_file_str = ""
      for img in validation_ds.img_files :
          validation_file_str += img + '\n'
      test_file_str = ""
      for img in test_ds.img_files :
          test_file_str += img + '\n'

      train_file = path.join(root_dir, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S") + '_train.log'
      validation_file = path.join(root_dir, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S") + '_validation.log'
      test_file = path.join(root_dir, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S") + '_test.log'

      with open(train_file, 'w') as f:
        f.write(train_file_str)
        f.close()
      with open(validation_file, 'w') as f:
        f.write(validation_file_str)
        f.close()
      with open(test_file, 'w') as f:
        f.write(test_file_str)
        f.close()

      csi_log2.save (save_file_name + '_test_result.log') # v3.3

      if root_dir2 is not None :
          train_file = path.join(root_dir2, 'log',s_file) + '_' + now.strftime("%m%d%H%M%S") + '_train.log'
          validation_file = path.join(root_dir2, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S") + '_validation.log'
          test_file = path.join(root_dir2, 'log', s_file) + '_' + now.strftime("%m%d%H%M%S") + '_test.log'
          with open(train_file, 'w') as f:
              f.write(train_file_str)
              f.close()
          with open(validation_file, 'w') as f:
            f.write(validation_file_str)
            f.close()
          with open(test_file, 'w') as f:
            f.write(test_file_str)
            f.close()
  return save_file_name

def load_model (model, file_path):
  model.load_state_dict(torch.load(file_path))
  csi_log.write( f"loaded model name : {file_path}\n" )
  return model

def load_test_model (model, loss_fn, file_path, test_dl):
  model = load_model (model, file_path)
  test(test_dl, model, loss_fn)
  return model



Using cuda device


In [4]:
# v3.3.1 : csi_log2.set_log_buf("")
# 1X test (PhaFall 2.3.2), previous LPD and csi_HiS instead of max abs

from pickle import TRUE
## main version 3.1,

import copy

def execute_cnn (execution_list, file_read_flag = False, nX_read_flag = False ):

    global train_ds, validation_ds, test_ds
    global n_epochs, n_batch_size, initial_lr, f_shuffle
    global csi_log, csi_log2

    csi_log.set_log_buf ("")
    csi_log.set_log_results ("")
    csi_log2.set_log_buf ("")
    csi_log2.set_log_results ("")

    csi_log.write(f"Processed on {device}...\n")
    i_count = 0;

    for ie in range( len(execution_list)) :
      i_count += 1
      print (f"Progress : [{i_count}/{len(execution_list)}]\n")

      e_list = execution_list[ie]
      n_epochs, n_batch_size, initial_lr, f_shuffle = hyper_parameter(e_list[HYPER])
      csi_log.write (f"******* {e_list[S_FN]}, Train({e_list[TRAIN]}), Test ({e_list[TEST]}), epoch ({n_epochs}), bs ({n_batch_size}), lr ({initial_lr}), shuffle({f_shuffle}) \n")

      if isinstance (e_list[CNN], list): # if it is a list (multiple CNN models for the above dataset)
        e_cnn =  e_list[CNN]
      else : # a sigle entry
        e_cnn =  [ e_list[CNN] ] # make it as list

      for im in range (len (e_cnn)) :  # there are multiple CNN models for the above dataset
          global Best_model, Best_loss, Best_epoch

          Best_model = None
          Best_loss = 1_000_000.0
          Best_epoch = 0

          csi_log.write(f"CNN ({cnn_model_name[e_cnn[im]]})\n")
          model, weights = cnn_model (e_cnn[im])

          loss_fn, optimizer, scheduler = loss_optimizer_scheduler (model, e_list[PARM])

          if weights is None :
              data_transf = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
          else :
            data_transf = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), weights.transforms()])

          if file_read_flag : # read image file names from the desiginated file
              #train_file = path.join(root_dir, 'log', e_list[TRAIN]) + '.log'

              train_ds = data_organize_from_file (e_list, data_transf,\
                      Train_log_file, search_flag=True, random_seed=e_list[R_SEED] )
              #validation_ds = data_organize_from_file (e_list, data_transf,\
              #        Validation_log_file, search_flag=False, random_seed=e_list[R_SEED] )
              validation_ds = data_organize_from_file (e_list, data_transf,\
                      Validation_log_file, search_flag=True, random_seed=e_list[R_SEED] )
              test_ds = data_organize_from_file (e_list, data_transf,\
                      Test_log_file, search_flag=False, random_seed=e_list[R_SEED] )
              csi_log.write (f"Train ({len(train_ds)}), Validation ({len(validation_ds)}), Test ({len(test_ds)})\n")
          else :

              if nX_read_flag :           # 3.0 updates
                  # split data into train_ds, validation_ds, test_ds from the desiginated directory
                  # compose temp_e_list
                  temp_e_list = copy.deepcopy(e_list)
                  temp_e_list[TRAIN][0] = e_list[TEST][0]
                  temp_e_list[TEST][0] = '' # path.join ('')
                  train_ds, validation_ds, test_ds, sep_result = data_organize(temp_e_list, data_transf,\
                                                random_seed=e_list[R_SEED], check_valid=True )
                  # save each train_da, validation_da, test_ds.img_files into temp files
                  train_file_str = ""
                  for img in train_ds.img_files :
                      train_file_str += img + '\n'
                  validation_file_str = ""
                  for img in validation_ds.img_files :
                      validation_file_str += img + '\n'
                  test_file_str = ""
                  for img in test_ds.img_files :
                      test_file_str += img + '\n'

                  with open("train_file", 'w') as f:
                    f.write(train_file_str)
                    f.close()
                  with open("validation_file", 'w') as f:
                    f.write(validation_file_str)
                    f.close()
                  with open("test_file", 'w') as f:
                    f.write(test_file_str)
                    f.close()
                  # release train_ds, validation_ds, then compose new train_da, validation_da
                  del train_ds, validation_ds
                  train_ds = data_organize_from_file (e_list, data_transf,\
                          "train_file", search_flag=True, random_seed=e_list[R_SEED] )
                  validation_ds = data_organize_from_file (e_list, data_transf,\
                        "validation_file", search_flag=False, random_seed=e_list[R_SEED] )

              else :
                  #r_seed = e_list[R_SEED]
                  train_ds, validation_ds, test_ds, sep_result = data_organize(e_list, data_transf,\
                                                  random_seed=e_list[R_SEED], check_valid=True )
                                                  #random_seed=(ie+1), check_valid=True )
              csi_log.write(f"random_seed ({e_list[R_SEED]}, nX_read_flag ({nX_read_flag})\n")

              if sep_result :
                csi_log.write (f"Train ({len(train_ds)}), Validation ({len(validation_ds)}), Test ({len(test_ds)})\n")
              else :
                csi_log.write ("data separation is NOT done correctly\n")

          # shuffle=True 로 지정했으므로, 모든 배치를 순회한 뒤 데이터가 섞임
          train_dl = DataLoader(train_ds, batch_size=n_batch_size, shuffle=f_shuffle)
          test_dl = DataLoader(test_ds, batch_size=n_batch_size, shuffle=f_shuffle)
          validation_dl = DataLoader(validation_ds, batch_size=n_batch_size, shuffle=f_shuffle)

          train_test(model, train_dl, validation_dl, test_dl, loss_fn, optimizer, scheduler)

          s_file = cnn_model_name[e_cnn[im]] + '_'+ e_list[S_FN] + \
              '_H' + str(e_list[HYPER]) + '_P' + str(e_list[PARM])+'_R' + str(e_list[R_SEED])
          s_file = save_model (model, s_file, root_dir, root_dir2, ds_save=True, model_save=False) # v3.2
          csi_log.set_log_buf("")
          csi_log2.set_log_buf("") # v3.3.1
          del model, loss_fn, optimizer, scheduler, train_ds, validation_ds, test_ds, train_dl, test_dl, validation_dl
          if device == "cuda" :
            torch.cuda.empty_cache()

      #if (i_count % 3) == 0 :
      #  csi_log.result("", flag=True)
      #  csi_log.set_log_results ("")

    csi_log.result("", flag=True) # print the all result strings, v3.1


In [6]:
random_seed = 31 # 31 or 42, 5
random_seed2 = 42
random_seed3 = 5

file_read_flag = False # Set True if the file separation is necessary, and Train_log_file, Validation_log_file, Test_log_file are
                      # used to get images from the file
nX_read_flag = False  # Set True if nX data is read after random split of the desinated directory (that is csi_img_dir2)
if nX_read_flag :
  file_read_flag = False

iteration_list = [ 1 ]

csi_img_dir_train = 'csi(1c)_AmFall_402_SDS_V0.10_O10_img_4.0'
csi_img_dir_test = 'csi(1c)_AmFall_402_SDS_V0.10_O10_img_4.0'

CNN_MODEL = CNN_SNU1

for iteration in iteration_list :

    execution_list=[
      [ [ path.join (csi_img_dir_train, 'enva'), path.join ('*','*','*.jpg') ],
        [ '', '' ],
        csi_img_dir_train+'_A_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed ]
      ,
      [ [ path.join (csi_img_dir_train, 'enva'), path.join ('*','*','*.jpg') ],
        [ '', '' ],
        csi_img_dir_train+'_A_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed2 ]
      ,
      [ [ path.join (csi_img_dir_train, 'enva'), path.join ('*','*','*.jpg') ],
        [ '', '' ],
        csi_img_dir_train+'_A_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed3 ]
    ]

    execute_cnn (execution_list, file_read_flag = file_read_flag, nX_read_flag = nX_read_flag  )

CNN_MODEL = CNN_R18

for iteration in iteration_list :

    execution_list=[
      [ [ path.join (csi_img_dir_train, 'envb'), path.join ('*','*','*.jpg') ],
        [ path.join (csi_img_dir_test, 'enva'), path.join ('*','*','*.jpg') ],
        csi_img_dir_train+'_B_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed ]
      ,
      [ [ path.join (csi_img_dir_train, 'envb'), path.join ('*','*','*.jpg') ],
        [ path.join (csi_img_dir_test, 'enva'), path.join ('*','*','*.jpg') ],
        csi_img_dir_train+'_B_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed2 ]
      ,
      [ [ path.join (csi_img_dir_train, 'envb'), path.join ('*','*','*.jpg') ],
        [ path.join (csi_img_dir_test, 'enva'), path.join ('*','*','*.jpg') ],
        csi_img_dir_train+'_B_A',
        [ CNN_MODEL, CNN_MODEL ],
        HYPER5, PARM0, random_seed3 ]
    ]

    execute_cnn (execution_list, file_read_flag = file_read_flag, nX_read_flag = nX_read_flag  )


Processed on cuda...
Progress : [1/3]

******* csi(1c)_AmFall_402_SDS_V0.10_O10_img_4.0_A_A, Train(['csi(1c)_AmFall_402_SDS_V0.10_O10_img_4.0/enva', '*/*/*.jpg']), Test (['', '']), epoch (40), bs (50), lr (0.0001), shuffle(True) 
CNN (SNU1)
Loss function (CrossEntropy) opt (Adam, 0.0001) lr_scheduler (MultiStepLR), [10, 20, 30], gamma(0.1)
random_seed (31, nX_read_flag (False)
Train (408), Validation (72), Test (120)
Epoch 1, lr(0.0001) : Avg loss(0.668605) : TP(29),FN(7),FP(1),TN(35) : Accuracy(0.88889), Precision(0.96667), Recall(0.80556), F1(0.87879) 
Epoch 2, lr(0.0001) : Avg loss(0.605515) : TP(36),FN(0),FP(19),TN(17) : Accuracy(0.73611), Precision(0.65455), Recall(1.00000), F1(0.79121) 
Epoch 3, lr(0.0001) : Avg loss(0.473646) : TP(36),FN(0),FP(3),TN(33) : Accuracy(0.95833), Precision(0.92308), Recall(1.00000), F1(0.96000) 
Epoch 4, lr(0.0001) : Avg loss(0.392534) : TP(36),FN(0),FP(2),TN(34) : Accuracy(0.97222), Precision(0.94737), Recall(1.00000), F1(0.97297) 
Epoch 5, lr(0.0001

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 232MB/s]


Loss function (CrossEntropy) opt (Adam, 0.0001) lr_scheduler (MultiStepLR), [10, 20, 30], gamma(0.1)
random_seed (31, nX_read_flag (False)
Train (306), Validation (54), Test (600)
Epoch 1, lr(0.0001) : Avg loss(0.244607) : TP(16),FN(11),FP(1),TN(26) : Accuracy(0.77778), Precision(0.94118), Recall(0.59259), F1(0.72727) 
Epoch 2, lr(0.0001) : Avg loss(0.159502) : TP(24),FN(3),FP(1),TN(26) : Accuracy(0.92593), Precision(0.96000), Recall(0.88889), F1(0.92308) 
Epoch 3, lr(0.0001) : Avg loss(0.280638) : TP(24),FN(3),FP(2),TN(25) : Accuracy(0.90741), Precision(0.92308), Recall(0.88889), F1(0.90566) 
Epoch 4, lr(0.0001) : Avg loss(1.014641) : TP(24),FN(3),FP(2),TN(25) : Accuracy(0.90741), Precision(0.92308), Recall(0.88889), F1(0.90566) 
Epoch 5, lr(0.0001) : Avg loss(0.384274) : TP(25),FN(2),FP(2),TN(25) : Accuracy(0.92593), Precision(0.92593), Recall(0.92593), F1(0.92593) 
Epoch 6, lr(0.0001) : Avg loss(0.495111) : TP(25),FN(2),FP(2),TN(25) : Accuracy(0.92593), Precision(0.92593), Recall(0.